In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

from nltk.corpus import stopwords
stop = list(set(stopwords.words('english')))

In [2]:
f_train = open('../data/train_14k_split_conll.txt','r',encoding='utf8')
line_train = f_train.readlines()

f_val = open('../data/dev_3k_split_conll.txt','r',encoding='utf8')
line_val = f_val.readlines()

f_test = open('../data/Hindi_test_unalbelled_conll_updated.txt','r',encoding='utf8')
line_test = f_test.readlines()

In [3]:
def get_data(lines):
    doc_count = 0
    i = 0
    #global train_word, train_word_type, train_doc_sentiment, train_uid, train_doc_id
    train_word = []
    train_word_type = []
    train_doc_sentiment = []
    train_uid = []
    train_doc_id = []

    while i < len(lines):
        '''
        if i < len(lines) and lines[i].split('\t')[0] in ['http','https']:
            i += 1
            while i < len(lines) and lines[i].split('\t')[0] != '/':
                i += 1
                
            i += 1    
        ''' 

        if i < len(lines) and lines[i].split('\t')[0] in ['@']:
            i += 2

        if i < len(lines):
            line = lines[i]
            line = line.replace('\n','').strip() #.lower()
            if line.split('\t')[0] == 'meta':
                doc_count += 1
                uid = int(line.split('\t')[1])
                sentiment = line.split('\t')[2]
                #if doc_count == 1575:
                #    print (uid)
                i += 1

            elif len(line.split('\t')) >= 2:
                if line.split('\t')[0] not in stop:
                    train_uid.append(uid)
                    train_doc_sentiment.append(sentiment)
                    train_word.append(line.split('\t')[0])
                    train_word_type.append(line.split('\t')[1])
                    train_doc_id.append(doc_count)

                i += 1

            else:
                i += 1

    train_df = pd.DataFrame()
    train_df['doc_id'] = train_doc_id
    train_df['word'] = train_word
    train_df['word_type'] = train_word_type
    train_df['uid'] = train_uid
    train_df['sentiment'] = train_doc_sentiment
    
    return train_df

In [4]:
def get_data_test(lines):
    doc_count = 0
    i = 0
    #global train_word, train_word_type, train_doc_sentiment, train_uid, train_doc_id
    train_word = []
    train_word_type = []
    train_uid = []
    train_doc_id = []

    while i < len(lines):
        '''
        if i < len(lines) and lines[i].split('\t')[0] in ['http','https']:
            i += 1
            while i < len(lines) and lines[i].split('\t')[0] != '/':
                i += 1
                
            i += 1    
        ''' 

        if i < len(lines) and lines[i].split('\t')[0] in ['@']:
            i += 2

        if i < len(lines):
            line = lines[i]
            line = line.replace('\n','').strip() #.lower()
            if line.split('\t')[0] == 'meta':
                doc_count += 1
                uid = int(line.split('\t')[1])
                #if doc_count == 1575:
                #    print (uid)
                i += 1

            elif len(line.split('\t')) >= 2:
                if line.split('\t')[0] not in stop:
                    train_uid.append(uid)
                    train_word.append(line.split('\t')[0])
                    train_word_type.append(line.split('\t')[1])
                    train_doc_id.append(doc_count)

                i += 1

            else:
                i += 1

    train_df = pd.DataFrame()
    train_df['doc_id'] = train_doc_id
    train_df['word'] = train_word
    train_df['word_type'] = train_word_type
    train_df['uid'] = train_uid
    
    return train_df

In [5]:
train_df = get_data(line_train)
val_df = get_data(line_val)
test_df = get_data_test(line_test)

In [6]:
train_word_types = train_df.groupby(['uid','word_type'])['word'].count().reset_index().pivot_table(index='uid',columns='word_type',values='word').reset_index().fillna(0)
val_word_types = val_df.groupby(['uid','word_type'])['word'].count().reset_index().pivot_table(index='uid',columns='word_type',values='word').reset_index().fillna(0)
test_word_types = test_df.groupby(['uid','word_type'])['word'].count().reset_index().pivot_table(index='uid',columns='word_type',values='word').reset_index().fillna(0)

In [7]:
train_texts = train_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index()
train_texts = pd.merge(train_texts,train_df[['uid','sentiment']],how='left').drop_duplicates().reset_index(drop=True)
train_texts.columns = ['uid','text','sentiment']
train_texts = pd.merge(train_texts,train_word_types,how='left').fillna(0)

val_texts = val_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index().reset_index(drop=True)
val_texts = pd.merge(val_texts,val_df[['uid','sentiment']],how='left').drop_duplicates()
val_texts.columns = ['uid','text','sentiment']
val_texts = pd.merge(val_texts,val_word_types,how='left').fillna(0)

test_texts = test_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index().reset_index(drop=True)
test_texts.columns = ['uid','text']
test_texts = pd.merge(test_texts,test_word_types,how='left').fillna(0)


In [8]:
train_texts.head(5)

,uid,text,sentiment,EMT,Eng,Hin,O
0,8,RT Love looks good Maddie !!! Ako lang ba yung...,neutral,0.0,16.0,4.0,3.0
1,14,@ Min _ Of _ Lyching @ manakgupta Mein kahna n...,neutral,0.0,6.0,9.0,7.0
2,26,Best luck sir World Cup ke liye bhot bhot subh...,positive,0.0,3.0,7.0,0.0
3,27,Yes . Great dialogues one . Also Chupke Chupke...,positive,0.0,11.0,7.0,10.0
4,33,@ TarekFatah Tu tere baap ke liye jo bola wo k...,negative,0.0,2.0,18.0,6.0


In [9]:
train_texts.head(5)

,uid,text,sentiment,EMT,Eng,Hin,O
0,8,RT Love looks good Maddie !!! Ako lang ba yung...,neutral,0.0,16.0,4.0,3.0
1,14,@ Min _ Of _ Lyching @ manakgupta Mein kahna n...,neutral,0.0,6.0,9.0,7.0
2,26,Best luck sir World Cup ke liye bhot bhot subh...,positive,0.0,3.0,7.0,0.0
3,27,Yes . Great dialogues one . Also Chupke Chupke...,positive,0.0,11.0,7.0,10.0
4,33,@ TarekFatah Tu tere baap ke liye jo bola wo k...,negative,0.0,2.0,18.0,6.0


In [10]:
def num_words(text):
    return len(text.split())

def uniq_words(text):
    return len(set(text.split()))

def len_char(text):
    return len(text)

def len_non_chars(text):
    return len(text) - len(re.sub("[^a-zA-Z0-9]", "",text))

def num_caps_words(text):
    return len([i for i in text.split() if i.capitalize() == i])

def num_all_caps(text):
    return len([i for i in text.split() if i.upper() == i])

def num_repeat_words(text):
    def word_extended(word):
        i = 1
        count = 0
        while i < len(word):
            if word[i] == word[i-1]:
                count += 1
            i += 1

        if count > 0:
            return 1
        else:
            return 0
        
    return sum([word_extended(i) for i in text.split()])

def avg_word_len(text):
    return np.mean([len(i) for i in text.split()])

In [11]:
train_texts['num_words'] = train_texts.text.apply(num_words)
train_texts['num_uniq_words'] = train_texts.text.apply(uniq_words)
train_texts['len_char'] = train_texts.text.apply(len_char)
train_texts['avg_word_len'] = train_texts.text.apply(avg_word_len)
train_texts['len_non_chars'] = train_texts.text.apply(len_non_chars)/train_texts['len_char']
train_texts['num_caps_words'] = train_texts.text.apply(num_caps_words)/train_texts['num_words']
train_texts['num_uniq_words'] = train_texts['num_uniq_words']/train_texts['num_words']
train_texts['num_all_caps'] = train_texts.text.apply(num_all_caps)/train_texts['num_words']
train_texts['num_repeat_words'] = train_texts.text.apply(num_repeat_words)/train_texts['num_words']

train_texts['EMT'] = train_texts['EMT']/train_texts['num_words']
train_texts['Eng'] = train_texts['Eng']/train_texts['num_words']
train_texts['Hin'] = train_texts['Hin']/train_texts['num_words']
train_texts['O'] = train_texts['O']/train_texts['num_words']

In [12]:
val_texts['num_words'] = val_texts.text.apply(num_words)
val_texts['num_uniq_words'] = val_texts.text.apply(uniq_words)
val_texts['len_char'] = val_texts.text.apply(len_char)
val_texts['avg_word_len'] = val_texts.text.apply(avg_word_len)
val_texts['len_non_chars'] = val_texts.text.apply(len_non_chars)/val_texts['len_char']
val_texts['num_caps_words'] = val_texts.text.apply(num_caps_words)/val_texts['num_words']
val_texts['num_uniq_words'] = val_texts['num_uniq_words']/val_texts['num_words']
val_texts['num_all_caps'] = val_texts.text.apply(num_all_caps)/val_texts['num_words']
val_texts['num_repeat_words'] = val_texts.text.apply(num_repeat_words)/val_texts['num_words']

val_texts['EMT'] = val_texts['EMT']/val_texts['num_words']
val_texts['Eng'] = val_texts['Eng']/val_texts['num_words']
val_texts['Hin'] = val_texts['Hin']/val_texts['num_words']
val_texts['O'] = val_texts['O']/val_texts['num_words']

In [13]:
test_texts['num_words'] = test_texts.text.apply(num_words)
test_texts['num_uniq_words'] = test_texts.text.apply(uniq_words)
test_texts['len_char'] = test_texts.text.apply(len_char)
test_texts['avg_word_len'] = test_texts.text.apply(avg_word_len)
test_texts['len_non_chars'] = test_texts.text.apply(len_non_chars)/test_texts['len_char']
test_texts['num_caps_words'] = test_texts.text.apply(num_caps_words)/test_texts['num_words']
test_texts['num_uniq_words'] = test_texts['num_uniq_words']/test_texts['num_words']
test_texts['num_all_caps'] = test_texts.text.apply(num_all_caps)/test_texts['num_words']
test_texts['num_repeat_words'] = test_texts.text.apply(num_repeat_words)/test_texts['num_words']

test_texts['EMT'] = test_texts['EMT']/test_texts['num_words']
test_texts['Eng'] = test_texts['Eng']/test_texts['num_words']
test_texts['Hin'] = test_texts['Hin']/test_texts['num_words']
test_texts['O'] = test_texts['O']/test_texts['num_words']

In [14]:
train_texts.head(10)

,uid,text,sentiment,EMT,Eng,Hin,O,num_words,num_uniq_words,len_char,avg_word_len,len_non_chars,num_caps_words,num_all_caps,num_repeat_words
0,8,RT Love looks good Maddie !!! Ako lang ba yung...,neutral,0.0,0.695652,0.173913,0.130435,23,1.000000,110,3.826087,0.245455,0.347826,0.217391,0.173913
1,14,@ Min _ Of _ Lyching @ manakgupta Mein kahna n...,neutral,0.0,0.272727,0.409091,0.318182,22,0.909091,102,3.681818,0.284314,0.545455,0.318182,0.136364
2,26,Best luck sir World Cup ke liye bhot bhot subh...,positive,0.0,0.300000,0.700000,0.000000,10,0.900000,53,4.400000,0.169811,0.300000,0.000000,0.000000
3,27,Yes . Great dialogues one . Also Chupke Chupke...,positive,0.0,0.392857,0.250000,0.357143,28,0.750000,124,3.464286,0.314516,0.785714,0.464286,0.142857
4,33,@ TarekFatah Tu tere baap ke liye jo bola wo k...,negative,0.0,0.076923,0.692308,0.230769,26,1.000000,116,3.500000,0.284483,0.269231,0.230769,0.153846
5,38,Desh bhakti baat wahi samajh sakte hai jo khud...,negative,0.0,0.130435,0.739130,0.130435,23,1.000000,118,4.173913,0.220339,0.260870,0.130435,0.130435
6,41,Madarchod mulle ye mathura Nahi dikha tha jab ...,negative,0.0,0.103448,0.758621,0.137931,29,0.931034,136,3.724138,0.242647,0.241379,0.137931,0.206897
7,48,Manya Pradhan Mantri mahoday Shriman Narendra ...,positive,0.0,0.136364,0.681818,0.181818,22,1.000000,129,4.909091,0.201550,0.636364,0.181818,0.136364
8,56,_ NSharif Kiya tum apne baap ki oulad nahi .??...,negative,0.0,0.111111,0.629630,0.259259,27,0.925926,122,3.555556,0.295082,0.370370,0.259259,0.148148
9,59,@ YouTube Are Tu aa rha h ki nhi Wo modi phir ...,neutral,0.0,0.178571,0.678571,0.142857,28,0.964286,121,3.357143,0.264463,0.321429,0.178571,0.178571


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler

In [16]:
vec = CountVectorizer(max_df=600,min_df=3,max_features=50000,stop_words='english')
vec.fit(train_texts.text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=600, max_features=50000, min_df=3,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [17]:
cv_train = vec.transform(train_texts.text)
cv_val = vec.transform(val_texts.text)
cv_test = vec.transform(test_texts.text)

In [18]:
[vec.get_feature_names()[i] for i in np.where(cv_train[0].toarray() > 0)[1]]

['ako',
 'ba',
 'good',
 'kasi',
 'lang',
 'looks',
 'masaya',
 'past',
 'sobrang',
 'sya',
 'yung']

In [24]:
lda = LatentDirichletAllocation(n_components=15)
lda.fit(cv_train)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=15, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [25]:
lda_train = lda.transform(cv_train)
lda_val = lda.transform(cv_val)
lda_test = lda.transform(cv_test)

In [26]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [27]:
print_top_words(lda,vec.get_feature_names(),5)

Topic #0: india kya minister tha gandhi
Topic #1: best like better miss think
Topic #2: bhai diya kar teri wo
Topic #3: jai ram shree sabka bharat
Topic #4: bahut desh ek jeet aapko
Topic #5: congress kya party tum janta
Topic #6: hain mubarak pakistan band allah
Topic #7: pakistan hoga mat toh tum
Topic #8: tere teri tu sale abe
Topic #9: big tum hy fan tco
Topic #10: pm allah congratulations narendra karta
Topic #11: world cup song wo super
Topic #12: thank really __ way like
Topic #13: hindu pe media muslim log
Topic #14: good happy birthday day bless



In [28]:
mm1 = MinMaxScaler((0,1))
mm2 = MinMaxScaler((0,1))
mm3 = MinMaxScaler((0,1))

In [29]:
mm1.fit(train_texts.num_words.values.reshape(-1,1))
mm2.fit(train_texts.len_char.values.reshape(-1,1))
mm3.fit(train_texts.avg_word_len.values.reshape(-1,1))

MinMaxScaler(copy=True, feature_range=(0, 1))

In [30]:
train_texts.num_words = mm1.transform(train_texts.num_words.values.reshape(-1,1))
train_texts.len_char = mm2.transform(train_texts.len_char.values.reshape(-1,1))
train_texts.avg_word_len = mm3.transform(train_texts.avg_word_len.values.reshape(-1,1))

val_texts.num_words = mm1.transform(val_texts.num_words.values.reshape(-1,1))
val_texts.len_char = mm2.transform(val_texts.len_char.values.reshape(-1,1))
val_texts.avg_word_len = mm3.transform(val_texts.avg_word_len.values.reshape(-1,1))

test_texts.num_words = mm1.transform(test_texts.num_words.values.reshape(-1,1))
test_texts.len_char = mm2.transform(test_texts.len_char.values.reshape(-1,1))
test_texts.avg_word_len = mm3.transform(test_texts.avg_word_len.values.reshape(-1,1))

In [31]:
train_texts.head(5)

,uid,text,sentiment,EMT,Eng,Hin,O,num_words,num_uniq_words,len_char,avg_word_len,len_non_chars,num_caps_words,num_all_caps,num_repeat_words
0,8,RT Love looks good Maddie !!! Ako lang ba yung...,neutral,0.0,0.695652,0.173913,0.130435,0.458333,1.000000,0.677419,0.263780,0.245455,0.347826,0.217391,0.173913
1,14,@ Min _ Of _ Lyching @ manakgupta Mein kahna n...,neutral,0.0,0.272727,0.409091,0.318182,0.437500,0.909091,0.625806,0.246261,0.284314,0.545455,0.318182,0.136364
2,26,Best luck sir World Cup ke liye bhot bhot subh...,positive,0.0,0.300000,0.700000,0.000000,0.187500,0.900000,0.309677,0.333472,0.169811,0.300000,0.000000,0.000000
3,27,Yes . Great dialogues one . Also Chupke Chupke...,positive,0.0,0.392857,0.250000,0.357143,0.562500,0.750000,0.767742,0.219846,0.314516,0.785714,0.464286,0.142857
4,33,@ TarekFatah Tu tere baap ke liye jo bola wo k...,negative,0.0,0.076923,0.692308,0.230769,0.520833,1.000000,0.716129,0.224183,0.284483,0.269231,0.230769,0.153846


In [32]:
test_texts.head(5)

,uid,text,EMT,Eng,Hin,O,num_words,num_uniq_words,len_char,avg_word_len,len_non_chars,num_caps_words,num_all_caps,num_repeat_words
0,1,Keh aese Rahe ho Jaise Pakistan wale Ni karte ...,0.0,0.125000,0.791667,0.083333,0.479167,0.958333,0.696774,0.254541,0.247788,0.500000,0.041667,0.000000
1,4,RT Anu's Prerna way ran saving ... And Ram Ram...,0.0,0.500000,0.250000,0.250000,0.395833,0.900000,0.638710,0.315257,0.365385,0.550000,0.200000,0.200000
2,17,Shukar hai ye pathan nae warna # PTM nay pr b ...,0.0,0.225806,0.580645,0.193548,0.625000,0.935484,0.832258,0.206555,0.291045,0.225806,0.193548,0.064516
3,18,Harsh pen decision . Didn't much option arm . ...,0.0,0.608696,0.130435,0.260870,0.458333,0.869565,0.735484,0.311297,0.268908,0.521739,0.304348,0.130435
4,34,RT I keep saying Kenyan rugby beautiful save K...,0.0,0.714286,0.095238,0.190476,0.416667,0.904762,0.670968,0.313811,0.256881,0.380952,0.285714,0.142857


In [33]:
trainX = np.hstack([train_texts.values[:,3:],lda_train])
valX = np.hstack([val_texts.values[:,3:],lda_val])
testX = np.hstack([test_texts.values[:,2:],lda_test])

trainy = train_texts.sentiment.values
valy = val_texts.sentiment.values

In [34]:
np.save('train_features',trainX)
np.save('val_features',valX)
np.save('test_features',testX)